In [1]:
%matplotlib widget
%load_ext autoreload
import numpy as np
import os 
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.figure, matplotlib.axes
import os
import sys
import numpy as np
import pickle as pkl
import yaml
import tqdm
thisfiledir=os.path.abspath("")
deepracingmodelsdir = os.path.abspath(os.path.join(thisfiledir, ".."))
deepracingdir = os.path.abspath(os.path.join(thisfiledir, "..", "..", "deepracing_py"))
if (not (deepracingmodelsdir in sys.path)) or (not (deepracingdir in sys.path)):
    sys.path = [deepracingmodelsdir, deepracingdir] + sys.path
import torch
import scipy.interpolate
mtrdir="/p/DeepRacing/MTR"
if (not (mtrdir in sys.path)):
    sys.path.insert(0, mtrdir)
print(sys.path)

['/p/DeepRacing/MTR', '/l/ttw2xk/deepracingws/src/deepracing/DCNN-Pytorch', '/l/ttw2xk/deepracingws/src/deepracing/deepracing_py', '/l/ttw2xk/deepracingws/src/deepracing/DCNN-Pytorch/plotting', '/p/DeepRacing/deepracingws/src/deepracing/DCNN-Pytorch', '/p/DeepRacing/deepracingws/src/deepracing/deepracing_py', '/l/ttw2xk/vcpkgexports/lambda_10_31_23/installed/x64-linux/lib/python311.zip', '/l/ttw2xk/vcpkgexports/lambda_10_31_23/installed/x64-linux/lib/python3.11', '/l/ttw2xk/vcpkgexports/lambda_10_31_23/installed/x64-linux/lib/python3.11/lib-dynload', '', '/l/ttw2xk/python3venvs/deepracing311/lib/python3.11/site-packages', '/l/ttw2xk/deepracingws/MTR', '/l/ttw2xk/source_builds/pytorch_cuda12', '/l/ttw2xk/python3venvs/deepracing311/lib/python3.11/site-packages/torchvision-0.16.0+fbb4cc5-py3.11-linux-x86_64.egg', '/l/ttw2xk/python3venvs/deepracing311/lib/python3.11/site-packages/torchaudio-2.1.0+6ea1133-py3.11-linux-x86_64.egg']


In [2]:
mixnet_results_dir = "/p/DeepRacing/mixnet_results"
mixnet_experiment = "agricultural_flue_8932"
with open(os.path.join(mixnet_results_dir, mixnet_experiment, "data.npz"), "rb") as f:
    mixnet_results_file = np.load(f)
    mixnet_results = {k: v.copy() for (k,v) in mixnet_results_file.items()}
print("Mixnet has %d points" % (mixnet_results["history"].shape[0],))

bezier_results_dir = "/p/DeepRacing/mixnet_bezier_results"
bezier_experiment = "widespread_beans_6059"
with open(os.path.join(bezier_results_dir, bezier_experiment, "data.npz"), "rb") as f:
    bezier_results_file = np.load(f)
    bezier_results = {k: v.copy() for (k,v) in bezier_results_file.items()}
print("Mixnet Bezier has %d points" % (bezier_results["history"].shape[0],))

composite_results_dir = "/p/DeepRacing/bamf_results"
composite_experiment = "sunny_coyote_3579"
with open(os.path.join(composite_results_dir, composite_experiment, "data.npz"), "rb") as f:
    composite_results_file = np.load(f, allow_pickle=True)
    composite_results = {k: v.copy() for (k,v) in composite_results_file.items()}
print("Composite model has %d points" % (composite_results["history"].shape[0],))


mtr_results_dir = "/p/DeepRacing/mtr_results/11_14_23"
mtr_experiment = "formal_pedestal_9890"
mtr_data_dir = "/p/DeepRacing/unpacked_datasets/local_fitting/v1/mtr_format/1second"
mtr_scenarios_dir = os.path.join(mtr_data_dir, "processed_scenarios_test")
mtr_sortfile = os.path.join(mtr_results_dir, mtr_experiment, "test_plots", "idx_sort.npz")
if not os.path.isfile(mtr_sortfile):
    with open(os.path.join(mtr_data_dir, "processed_scenarios_test_infos.pkl"), "rb") as f:
        mtr_infos = pkl.load(f)
    mtr_keys = mtr_infos[0].keys()
    entries = []
    for (idx, info) in tqdm.tqdm(enumerate(mtr_infos), total=len(mtr_infos)):
        scenario_id = info["scenario_id"]
        with open(os.path.join(mtr_scenarios_dir, scenario_id+".metadata.yaml"), "r") as f:
            scenario_metadata = yaml.safe_load(f)
        deepracing_dir = os.path.dirname(scenario_metadata["deepracing_file"])
        dset_index = scenario_metadata["index"]
        car_index = int(os.path.basename(deepracing_dir).split("_")[-1])
        dated_trackname : str = os.path.basename(os.path.dirname(deepracing_dir))
        trackname = dated_trackname.split("_")[0]
        entries.append((idx, scenario_id, trackname, car_index, dset_index))
    entries_sorted = sorted(entries, key=lambda entry : (entry[2], entry[3], entry[4]))
    scenario_ids_sorted = np.asarray([e[1] for e in entries_sorted], dtype=object)
    idx_sort = np.asarray([e[0] for e in entries_sorted], dtype=np.int64)
    with open(mtr_sortfile, "wb") as f:
        np.savez(f, idx_sort=idx_sort, scenario_ids=scenario_ids_sorted)

with open(mtr_sortfile, "rb") as f:
    npfile = np.load(f, allow_pickle=True)
    idx_sort = npfile["idx_sort"].copy()
    scenario_ids_sorted = npfile["scenario_ids"].copy()
with open(os.path.join(mtr_scenarios_dir, scenario_ids_sorted[1]+".metadata.yaml"), "r") as f:
    print(yaml.safe_load(f))

with open(os.path.join(mtr_results_dir, mtr_experiment, "test_plots", "data.npz"), "rb") as f:
    mtr_results_file = np.load(f, allow_pickle=True)
    mtr_results = {k: mtr_results_file[k][idx_sort].copy() for k in mtr_results_file.keys()}
print("MTR has %d points" % (mtr_results["predictions"].shape[0],))



Mixnet has 126577 points
Mixnet Bezier has 126577 points
Composite model has 126577 points
{'deepracing_file': '/p/DeepRacing/unpacked_datasets/local_fitting/v1/deepracing_standard/Bahrain_7_13_2023_15_43_22_trajectory_data/car_0/test_data.npz', 'index': 1}
MTR has 126577 points


In [5]:
# idx_samp = 15
# best_curve_idx = mtr_results["best_curve_idx"]
# mtr_predictions = np.stack([mtr_results["predictions"][i, best_curve_idx[i]] for i in range(best_curve_idx.shape[0])], axis=0)
# comp_samp = composite_results["predictions"][idx_samp,1:]
# gt_samp = composite_results["ground_truth"][idx_samp,1:]
# mtr_samp = mtr_predictions[idx_samp]


In [4]:
matplotlib.use("module://ipympl.backend_nbagg")